In [ ]:
# default_exp utils

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nbdev.test import test_eq
from nbdev.showdoc import *

# Utils
> Useful functiosn to reshape/arrange/reduce raw data into clean data to add to the record

In [ ]:
#export
import numpy as np
from typing import Dict, Tuple, Sequence, Union, Callable
import scipy.interpolate as interpolate
from scipy.ndimage import convolve1d

from theonerig.core import *

In [ ]:
#export
def extend_sync_timepoints(timepoints:np.ndarray, signals:np.ndarray, 
                           up_bound, low_bound=0) -> Tuple[DataChunk, DataChunk]:
    """From `timepoints` and `signals` list, extend it on the left so it includes `low_bound`, and extend it
    up to `up_bound`.
    Return the new timepoints and signals as DataChunk objets with the number of timepoints added to the left
    as a tuple.
    """
    assert len(timepoints) == len(signals)
    timepoints = np.array(timepoints)
    signals = np.array(signals)
    spb = np.mean(timepoints[1:]-timepoints[:-1]) #spf: sample_per_bin
        
    #Left and right side are just prolongation of the sample_times up 
    # from (0-sample_per_fr) to (len+sample_per_fr) so it covers all timepoints
    left_side  = np.arange(timepoints[0]-spb , low_bound - spb, -spb)[::-1].astype(int)
    right_side = np.arange(timepoints[-1]+spb,  up_bound + spb,  spb).astype(int)

    new_timepoints = np.concatenate((left_side, 
                                     timepoints, 
                                     right_side))
    
    timepoint_chunk = DataChunk(data=new_timepoints, idx=0, group="sync")
    signal_chunk    = DataChunk(data=signals, idx=len(left_side), group="sync")
    return (timepoint_chunk, signal_chunk, len(left_side))

It extends the timepoints by finding the typical distance between timepoints so it includes both left and right limits.

In [ ]:
extended_tp, extended_sig, n_left = extend_sync_timepoints([7900,8900,9900], signals=[1,2,3], up_bound=15000)

In [ ]:
print(extended_tp, end="\n\n")
print(extended_sig, end="\n\n")
print("N timepoints added to the left:",n_left)

In [ ]:
#export
def align_sync_timepoints(timepoints:np.ndarray, signals:np.ndarray,
                          ref_timepoints:DataChunk, ref_signals:DataChunk, 
                          shift=None) -> DataChunk:
    """Align the `signals` of a `timepoints` timeserie to a reference `ref_timepoints` with the corresponding
    `ref_signals`. A `shift` can be directly specified, otherwise it will be searched by finding the maximum
    of the correlations of the two signals timeseries.
    Returns a DataChunk of the aligned timepoints"""
    assert len(timepoints) == len(signals)
    timepoints = np.array(timepoints)
    signals = np.array(signals)
    
    if shift is None: #If a shift is provided we use it, otherwise we use the max correlation
        shift = np.argmax(np.correlate(ref_signals, signals, mode="valid"))
        
    spb = np.mean(timepoints[1:]-timepoints[:-1]) #spf: sample_per_bin
    n_left  = ref_signals.idx + shift
    n_right = (len(ref_timepoints) 
               - len(timepoints) 
               - n_left)

    init_left  = timepoints[0]-spb
    init_right = timepoints[-1]+spb

    left_side  = np.arange(init_left , init_left-(spb*n_left+1), -spb)[:n_left][::-1].astype(int)
    right_side = np.arange(init_right, init_right+(spb*n_right+1), spb)[:n_right].astype(int)

    new_timepoints = np.concatenate((left_side, 
                                     timepoints, 
                                     right_side))
    return DataChunk(data=new_timepoints, idx=0, group="sync")

In [ ]:
#Aligning these signals 1,2,3 to the previously extended timepoints and signals
aligned_timepoints = align_sync_timepoints(timepoints=[1000,2000,3000], signals=[1,2,3],
                     ref_timepoints=extended_tp, ref_signals=extended_sig)

#We can observe 8 timepoints added to our function to match all existing frame of the reference.
print(aligned_timepoints)
test_eq(len(aligned_timepoints), len(extended_tp))

In [ ]:
#export
def resample_to_timepoints(timepoints:np.ndarray, data:np.ndarray, 
                             ref_timepoints:DataChunk, group="data") -> DataChunk:
    """Resample the `data` at the `timepoints` to an array at the timepoints of `ref_timepoints`.
    Return a DataChunck of the resampled data belonging to `group`."""
    
    assert len(timepoints) == len(data)
    timepoints = np.array(timepoints)
    data = np.array(data)
    
    start_idx = np.argmax(timepoints[0] <=ref_timepoints)
    stop_idx  = np.argmax(timepoints[-1]<=ref_timepoints)
    
    if len(ref_timepoints[start_idx:stop_idx]) < len(timepoints): #Downsampling
        distance = (np.argmax(timepoints>ref_timepoints[start_idx+1]) 
                - np.argmax(timepoints>ref_timepoints[start_idx]))
    
        kernel = np.ones(distance)/distance
        data = convolve1d(data, kernel, axis=0) #Smooting to avoid weird sampling

    new_data = interpolate.interp1d(timepoints, data, axis=0)(ref_timepoints[start_idx:stop_idx])

    idx = ref_timepoints.idx + start_idx
    return DataChunk(data=new_data, idx = idx, group=group)

In [ ]:
import matplotlib.pyplot as plt
#Downsampling
np.random.seed(3)
len_data = 40
wrong_dim_data = np.random.rand(len_data)
wrong_dim_timepoints = np.linspace(0,10000,len_data,endpoint=False, dtype=int)

downsamp_data = resample_to_timepoints(wrong_dim_timepoints, wrong_dim_data, 
                                         ref_timepoints=extended_tp)

plt.figure(figsize=(10,4))
plt.plot(wrong_dim_timepoints, wrong_dim_data)
start,stop = downsamp_data.idx, downsamp_data.idx+len(downsamp_data)
plt.plot(extended_tp[start:stop], downsamp_data)
print("Orig len:", len(wrong_dim_data), "Target len:",len(extended_tp[start:stop]))

In [ ]:
#Upsampling
np.random.seed(3)
len_data = 5
wrong_dim_data = np.random.rand(len_data)
wrong_dim_timepoints = np.linspace(0,10000,len_data,endpoint=False, dtype=int)

downsamp_data = resample_to_timepoints(wrong_dim_timepoints, wrong_dim_data, 
                                         ref_timepoints=extended_tp)

plt.figure(figsize=(10,4))
plt.plot(wrong_dim_timepoints, wrong_dim_data)
start,stop = downsamp_data.idx, downsamp_data.idx+len(downsamp_data)
plt.plot(extended_tp[start:stop], downsamp_data)
print("Orig len:", len(wrong_dim_data), "Target len:",len(extended_tp[start:stop]))

In [ ]:
#export
def stim_to_dataChunk(stim_values, stim_start_idx, reference:DataChunk) -> DataChunk:
    """Factory function for DataChunk of a stimulus"""
    return DataChunk(data=stim_values, idx = (reference.idx + stim_start_idx), group="stim")

In [ ]:
#export
def spike_to_dataChunk(spike_timepoints, ref_timepoints:DataChunk) -> DataChunk:
    """`spike_timepoints` must be a dictionnary of cell spike_timepoints list. This function then
    bins the """
    cell_keys = list(map(str, 
                         sorted(map(int, 
                                    spike_timepoints.keys()))))
    cell_map = dict([ (cell_key, i) for i, cell_key in enumerate(cell_keys) ])
    spike_bins = np.zeros((ref_timepoints.shape[0], len(cell_keys)))
    bins = np.concatenate((ref_timepoints[:], [(ref_timepoints[-1]*2)-ref_timepoints[-2]]))
    for i, cell in enumerate(cell_keys):
        
        spike_bins[:, i] = np.histogram(spike_timepoints[cell], bins)[0]
        
    datachunk = DataChunk(data=spike_bins, idx = ref_timepoints.idx, group="cell")
    datachunk.attrs["cell_map"] = cell_map
    return datachunk

In [ ]:
#export
def parse_stim_args(stim_name, stim_ref):
    """Function really specific to Asari Lab stimuli. Stimuli were stored as h5 files. This function parse
    the attributes of the stimuli that were stored in the h5 references of the stimuli."""
    args = {}
    if stim_name in ["chirp_am", "chirp_fm", "chirp_co"]:
        #+ add on off timings at the beginning?
        args["n_repeat"] = int(stim_ref.attrs["n_repeat"])
    if stim_name in ["chirp_fm"]:
        args["max_freq"] = int(stim_ref.attrs["max_frequency"])
    if stim_name in ["moving_gratings"]:
        #+ Convert to degree units
        args["n_fr_stim"] = int(stim_ref.attrs["n_frame_on"])#.keys()
        args["n_fr_interstim"] = int(stim_ref.attrs["n_frame_off"])
        args["n_repeat"] = int(stim_ref.attrs["n_repeat"])
        args["n_angle"] = int(stim_ref.attrs["n_angle"])
        args["sp_freqs"] = list(map(int,stim_ref.attrs["spatial_frequencies"][1:-1].split(",")))
        args["speeds"] = list(map(int,stim_ref.attrs["speeds"][1:-1].split(",")))
    if stim_name in ["flickering_bars", "checkerboard", "flickering_bars_pr"]:
        #Get the size of the sides in angle
        pass
    return args

In [ ]:
#export
def peak_sta_frame(sta):
    abs_sta   = np.abs(sta)
    idx_frame = np.unravel_index(abs_sta.argmax(), sta.shape)[0]
    return sta[idx_frame]

def stim_inten_norm(stim_inten):
    stim_inten = stim_inten.astype(float)
    stim_inten -= np.min(stim_inten)
    stim_inten -= np.max(stim_inten)/2
    stim_inten /= np.max(np.abs(stim_inten))
    return np.round(stim_inten, 2)
#     stim_inten[stim_inten==255] = 1
#     stim_inten[stim_inten==0]   = -1
#     stim_inten[(stim_inten==127) | (stim_inten==128)] = 0 #In case

In [ ]:
#export
def twoP_dataChunks(ref_timepoints:DataChunk, frame_timepoints, len_epochs, C_matrix, S_matrix):
    C_datachunk_l = []
    S_datachunk_l = []
    cursor = 0
    for i, len_epoch in enumerate(len_epochs):
        start_idx = np.argmax(ref_timepoints>frame_timepoints[i][0])
        stop_idx  = np.argmax(ref_timepoints>frame_timepoints[i][-6])
        sub_C, sub_S = C_matrix.T[cursor:cursor+len_epoch], S_matrix.T[cursor:cursor+len_epoch]
        cursor += len_epoch
        f = interpolate.interp1d(range(len_epoch), sub_C, axis=0)
        C_datachunk_l.append(DataChunk(data=f(np.linspace(0,len_epoch-1,stop_idx-start_idx)), 
                                       idx=start_idx, 
                                       group="cell"))
        f = interpolate.interp1d(range(len_epoch), sub_S, axis=0)
        S_datachunk_l.append(DataChunk(data=f(np.linspace(0,len_epoch-1,stop_idx-start_idx)), 
                                       idx=start_idx, 
                                       group="cell"))
    return (C_datachunk_l, S_datachunk_l)

In [ ]:
#export
def img_2d_fit(shape, param_d, f):
    y_, x_ = shape
    xy = np.meshgrid(range(x_), range(y_))
    return f(xy, **param_d).reshape(y_,x_)

In [ ]:
#export
def fill_nan(A):
    '''
    interpolate to fill nan values. BRYAN WOODS@StackOverflow
    '''
    inds = np.arange(A.shape[0])
    good = np.where(np.isfinite(A))
    f = interpolate.interp1d(inds[good], A[good],bounds_error=False)
    B = np.where(np.isfinite(A),A,f(inds))
    return B

In [ ]:
from nbdev.export import *
notebook2script()